In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as lib
import numpy.matlib as mat
import math as math
import warnings
import h5py
from itertools import product
# Mount Google Drive to Colab
#from google.colab import drive
#drive.mount('/content/drive')


### Clases

In [ ]:
#   IN:
#       domain:     Space domain. A 6x1 vector containing:
#                     - x-coordinate of the lower left corner (m)
#                     - y-coordinate of the lower left corner (m)
#                     - z-coordinate (lower)
#                     - x-coordinate of the upper right corner (m)
#                     - y-coordinate of the upper right corner (m)
#                     - z-coordinate (upper)
#                     - Number of cells in the x direction
#                     - Number of cells in the y direction
#                     - Number of cells in the z direction

class Domain:
  def __init__(self, x_low, y_low, z_low, x_up, y_up, z_up, x_cells, y_cells, z_cells):
    self.x_low = x_low
    self.y_low = y_low
    self.z_low = z_low
    self.x_up = x_up
    self.y_up = y_up
    self.z_up = z_up
    self.x_cells = x_cells
    self.y_cells = y_cells
    self.z_cells = z_cells


#     params:     Parameter struct containing the physical parameters as
#                  given in the following table:
#
#                   VARIABLE  | UNITS         | DESCRIPTION
#                  -----------+---------------+-------------------------------------------------------
#                  0 IN       |  g·m-2·day-1  |  Input rate of nutrient
#                  1 mu       |  day-1        |  Maximum growth rate of phytoplankton
#                  2 k        |               |  Nutrient content in phytoplankton
#                  3 mN       |  day-1        |  Loss rate of nutrient
#                  4 fP       |  g·m-2·day-1  |  Maximum feeding rate of zooplankton on phytoplankton
#                  5 HN       |  g·m-2        |  Half-saturation constant of nutrient
#                  6 HP       |  g·m-2        |  Half-saturation constant of phytoplankton
#                  7 d        |  ???          |  Diffusion coefficient
#                  8 vmax     |  m/s          |  Maximum fluid velocity
#                  9 A        |  %            |  Amplitude as a percentage of distance from
#                             |               |  stationary point to origin (if iCondType == 'scaled')

class Params:
  def __init__(self, IN, mu, k , mN, fP, HN, HP, d, vmax, A):
    self.IN = IN
    self.mu = mu
    self.k = k
    self.mN = mN
    self.fP = fP
    self.HN = HN
    self.HP = HP
    self.d = d
    self.vmax = vmax
    self.A = A

#       eddy:       Eddy currents. Struct containing the following fields as Nx1
#                   vectors, where N is the number of eddies of the simulation.
#                   NOTE: Mirror eddies will be created internally to enforce
#                   periodic boundary conditions.
#
#                   VARIABLE  | UNITS         | DESCRIPTION
#                  -----------+---------------+--------------------------------------------------------
#                    x        |  m            |  x-coordinate of the center of the eddies
#                    y        |  m            |  y-coordinate of the center of the eddies
#                    z        |  m            |  z-coordinate of the center of the eddies
#                    r        |  m            |  Radius of the eddies
#                    s_z      |               |  Rotation directon (1: clockwise, -1: counter-clockwise) perpendicular to plane xy
#                    s_y      |               |  Rotation directon (1: clockwise, -1: counter-clockwise) perpendicular to plane xz
#                    s_x      |               |  Rotation directon (1: clockwise, -1: counter-clockwise) perpendicular to plane yz

class Eddy:
  def __init__(self, x, y, z, r, s_z, s_y, s_x):
    self.x = x
    self.y = y
    self.z = z
    self.r = r
    self.s_z = s_z
    self.s_y = s_y
    self.s_x = s_x

### Parámetros de la Simulación

In [ ]:
nEddy=100

domain = Domain(x_low=-25, y_low=-25, z_low=-25, x_up=25, y_up=25, z_up=25, x_cells=60, y_cells=60, z_cells=60)
#Size of domain
Lx = domain.x_up - domain.x_low
Ly = domain.y_up - domain.y_low
Lz = domain.z_up - domain.z_low
#Space steps
dx = Lx / domain.x_cells
dy = Ly / domain.y_cells
dz = Lz / domain.z_cells

print('dx',dx)
print('dy',dy)
print('dz',dz)



params = Params(IN=0.00225, mu=0.5, k=0.01 , mN=0.015, fP=1.8, HN=0.005, HP=4.0, d=0.09, vmax=0.3, A=0.5)
np.random.seed(1)
eddy = Eddy(x=Lx*np.random.rand(nEddy, 1)+domain.x_low, y=Ly*np.random.rand(nEddy, 1)+domain.y_low, z=Lz*np.random.rand(nEddy, 1)+domain.z_low,
            r=abs(np.random.normal(20,5,(nEddy,1))),s_z=2*(np.round(np.random.rand(nEddy, 1))-0.5),s_y=2*(np.round(np.random.rand(nEddy, 1))-0.5),s_x=2*(np.round(np.random.rand(nEddy, 1))-0.5))

# Create grid
x, y, z = np.meshgrid(np.linspace(domain.x_low, domain.x_up, domain.x_cells),
                      np.linspace(domain.y_low, domain.y_up, domain.y_cells),
                      np.linspace(domain.z_low, domain.z_up, domain.z_cells),
                      indexing='ij')

#Inital cond

iCondN = 2+params.A*(np.sin(math.pi/Lx*2*x)+np.sin(math.pi/Lz*2*z))
iCondP = 2+params.A*(np.sin(math.pi/Ly*2*y)+np.sin(math.pi/Lz*2*z))

print('iCondP is in ', iCondP.min(),'-' ,iCondP.max(),'con media' ,np.mean(iCondP))


NombreSim='noperiodico_sindensidades_(dvnoperiodica)_pruebadifusion'

#Time parameters
#60 seg = 1 min = 0,000694444 días
# tSpan = np.array([0, 0.1, 80])
#xc = 0; yc = 0; sigma = 5000;
#iCondP = (exp(-((x-xc).^2 + (y-yc).^2)./(2*sigma)));

#iCondN = (np.sin(math.pi/(Lx/2)*(x-100)))*(np.sin(math.pi/(Ly/2)*(y-0)))
#iCondP = np.sqrt(x**2+y**2)

In [ ]:
#Time parameters
#60 seg = 1 min = 0,000694444 días
steptiempo=0.1*24*3600   # seg
v=params.vmax*params.mu/(24*3600)
print("Courant–Friedrichs–Lewy condition:")
print("Para step de ",steptiempo,"segundo/s y velocidad",v,"m/s C=",steptiempo*v*(1/dx+1/dy+1/dz))



tSpan = np.array([0, 0.1, 120]) #dias
RecordStep=1

### Funciones

In [ ]:
  def PsiFun(x,y,z):
    s=1

    psi_1 = np.zeros_like(x)
    psi_2 = np.zeros_like(x)
    psi_3 = np.zeros_like(x)


    for i in range(x.shape[0]):
      for j in range(x.shape[1]):
        for k in range(x.shape[2]):


          psi_1[i][j][k] = s*np.sum(eddy.s_z*np.exp(-((x[i][j][k]-eddy.x)**2+(y[i][j][k]-eddy.y)**2)/eddy.r**2))

          psi_2[i][j][k] = s*np.sum(eddy.s_y*np.exp(-((x[i][j][k]-eddy.x)**2+(z[i][j][k]-eddy.z)**2)/eddy.r**2))

          psi_3[i][j][k] = s*np.sum(eddy.s_x*np.exp(-((y[i][j][k]-eddy.y)**2+(z[i][j][k]-eddy.z)**2)/eddy.r**2))




    return psi_1, psi_2, psi_3


  def Dx(A):

    #Derivada central en el eje x 

    dA = (np.roll(A, -1, axis=0) - np.roll(A, 1, axis=0)) / (2 * dx)

    return dA

  def Dy(A):

    #Derivada central en el eje y 

    dA = (np.roll(A, -1, axis=1) - np.roll(A, 1, axis=1)) / (2 * dy)

    return dA

  def Dz(A):

    #Derivada central en el eje z 

    dA = (np.roll(A, -1, axis=2) - np.roll(A, 1, axis=2)) / (2 * dz)

    return dA

  def Dz_Neumann(A):

    dA = np.zeros_like(A)

    # Diferencia central para los puntos internos
    dA[:, :, 1:-1, ...] = (A[:, :, 2:, ...] - A[:, :, :-2, ...]) / (2 * dz)

    # Neumann primer punto
    dA[:, :, 0, ...] = (A[:, :, 1, ...] - A[:, :, 1, ...]) / (2 * dz)

    # Neumann segundo punto
    dA[:, :, -1, ...] = (A[:, :, -2, ...] - A[:, :, -2, ...]) / (2 * dz)
    
    return dA

  def DDx(A):

            # Calcula la segunda derivada central en x
    ddA = (np.roll(A, -1, axis=0) - 2 * A + np.roll(A, 1, axis=0)) / (dx ** 2)

    return ddA

  def DDy(A):

            # Calcula la segunda derivada central en y
    ddA = (np.roll(A, -1, axis=1) - 2 * A + np.roll(A, 1, axis=1)) / (dy ** 2)

    return ddA

  def DDz(A):
        # Calcula la segunda derivada central en z
    ddA = (np.roll(A, -1, axis=2) - 2 * A + np.roll(A, 1, axis=2)) / (dz ** 2)

    return ddA

  def DDz_Neumann(A):
    ddA = np.zeros_like(A)

    # Diferencia central para los puntos internos
    ddA[:, :, 1:-1, ...] = (A[:, :, 2:, ...] - 2 * A[:, :, 1:-1, ...] + A[:, :, :-2, ...]) / (dz ** 2)

    # Neumann primer punto
    ddA[:, :, 0, ...] = (A[:, :, 1, ...] - 2 * A[:, :, 0, ...] + A[:, :, 1, ...]) / (dz ** 2)

    # Neumann último punto
    ddA[:, :, -1, ...] = (A[:, :, -2, ...] - 2 * A[:, :, -1, ...] + A[:, :, -2, ...]) / (dz ** 2)
    
    return ddA


  def DnFun(n,p):
    # dn = d*(DDx(n)+DDy(n)) - n.*dv - (vx.*Dx(n)+vy.*Dy(n)) + in - a*n./(1+n).*p - mn*n;  Esto es en 2D
    dn = 0;
    dn = dn - n*dv - (vx*Dx(n)+vy*Dy(n)+vz*Dz(n))              # Advection
    dn = dn + params.d*(DDx(n)+DDy(n)+DDz(n))                  # Diffusion
    dn = dn + IN - a*n/(1+n)*p - mn*n                          # Reaction

    return dn

  def DpFun(n,p):
    #    dp = d*(DDx(p)+DDy(p)) - p.*dv - (vx.*Dx(p)+vy.*Dy(p)) + n./(1+n).*p - fp*p./(1+p); Esto es en 2D
    dp = 0
    dp = dp - p*dv - (vx*Dx(p)+vy*Dy(p)+vz*Dz(p))                          # Advection
    dp = dp + params.d*(DDx(p)+DDy(p)+DDz(p))                              # Diffusion
    dp = dp + n/(1+n)*p - fp*p/(1+p)                                       # Reaction



    return dp

  def adjustIC(iCondN, iCondP):
    err = np.zeros((2, 2))              ######### BUSCAR BIEN QUE AJUSTE SE HACE, POR SI ACASO SE
    c1 = 1
    c2 = (a*fp-IN-a+mn)/mn
    c3 = (a*fp-IN)/mn
    n1 = (-c2+math.sqrt(c2**2-4*c1*c3))/(2*c1)
    p1 = fp/n1+fp-1;

    if n1 < 0 or p1 < 0:
      err[0][:]= [n1,p1]
      n1 = (-c2-math.sqrt(c2**2-4*c1*c3))/(2*c1)
      p1 = fp/n1+fp-1

    if n1 < 0 or p1 < 0:
      err[1][:]= [n1,p1]
      #print('err')
      #raise ValueError('The given parameters do not have a valid stationary point')
      warnings.warn('The given parameters do not have a valid stationary point')
    #Transform to dimensionless space
    iCondn = iCondN / params.HN
    iCondp = iCondP / params.HP

    #Translate so that the mean is centered at the stationary point
    iCondn = iCondn + n1 - np.mean(iCondn)
    iCondp = iCondp + p1 - np.mean(iCondp)

    #Scale range keeping the mean if there are negative values
    if np.any(iCondn <= 0):
      iCondn = (iCondn - n1)/(n1-np.min(iCondn))*(params.A*n1)+n1

    if np.any(iCondp <= 0):
      iCondp = (iCondp - p1)/(p1-np.min(iCondp))*(params.A*p1)+p1

    # Display adjustment in physical space
    iCondN = iCondn * params.HN; iCondP = iCondp * params.HP;
    print(f"iCondN adjusted to [{np.min(iCondN):.6f}, {np.max(iCondN):.6f}] with mean {np.mean(iCondN):.6f}")
    print(f"iCondP adjusted to [{np.min(iCondP):.6f}, {np.max(iCondP):.6f}] with mean {np.mean(iCondP):.6f}")

    # Error checking
    if abs(np.mean(iCondn) - n1) > 1e-10:
      warnings.warn(f'Something went wrong in adjustment. Mean of iCondn does not match stationary point n1 ({np.mean(iCondn):.6f} vs. {n1:.6f})')

    if abs(np.mean(iCondp) - p1) > 1e-10:
      warnings.warn(f'Something went wrong in adjustment. Mean of iCondp does not match stationary point p1 ({np.mean(iCondp):.6f} vs. {p1:.6f})')

    return iCondn, iCondp


  def RK4(n,p):

    k1 = dt*DnFun(n,p)
    m1 = dt*DpFun(n,p)

    k2 = dt*DnFun(n+k1/2, p+m1/2)
    m2 = dt*DpFun(n+k1/2, p+m1/2)

    k3 = dt*DnFun(n+k2/2, p+m2/2)
    m3 = dt*DpFun(n+k2/2, p+m2/2)

    k4 = dt*DnFun(n+k3, p+m3)
    m4 = dt*DpFun(n+k3, p+m3)

    n = n + (k1+(k2*2)+(k3*2)+k4)/6
    p = p + (m1+(m2*2)+(m3*2)+m4)/6

    return n,p
def GenEddy():
    
    offx = Lx + dx
    offy = Ly + dy

    # Inversiones posiciones en z
    z_sup = 2 * domain.z_up - eddy.z  
    z_inf = 2 * domain.z_low - eddy.z  

    
    eddylistx = [eddy.x, eddy.x + offx, eddy.x - offx]  
    eddylisty = [eddy.y, eddy.y + offy, eddy.y - offy]  
    eddylistz = [eddy.z, z_sup, z_inf]                 

    #  27 combinaciones
    comb = list(product(eddylistx, eddylisty, eddylistz))
    X_comb, Y_comb, Z_comb = [], [], []
    for i in range(len(comb)):
        X_comb.append(comb[i][0])
        Y_comb.append(comb[i][1])
        Z_comb.append(comb[i][2])

    
    eddy.x = np.vstack(X_comb)
    eddy.y = np.vstack(Y_comb)
    eddy.z = np.vstack(Z_comb)
    
    # Si z_sup o z_inf invierte signos
    signs = []
    for i in range(len(comb)):
        _, _, z_part = comb[i]
        # Flip signs for z-mirrored eddies (z_sup or z_inf)
        if z_part is eddylistz[1] or z_part is eddylistz[2]:
            signs.extend([-1] * nEddy)
        else:
            signs.extend([1] * nEddy)
    signs = np.array(signs).reshape(-1, 1)

    # inversiones de rotacion
    eddy.s_y = mat.repmat(eddy.s_y, 27, 1) * signs
    eddy.s_x = mat.repmat(eddy.s_x, 27, 1) * signs

    # Sin inversion
    eddy.s_z = mat.repmat(eddy.s_z, 27, 1)
    eddy.r = mat.repmat(eddy.r, 27, 1)  
    
   

    if x.shape!= y.shape or x.shape!= z.shape:    
      print('x, y, z  must have the same size')
      print('x: ', x.shape)
      print('y: ', y.shape)
      print('z: ', z.shape)
      exit()

    # Get preliminary stream field and velocities

    psi_1,psi_2,psi_3 = PsiFun(x,y,z)


    #Velocities with Stream function in plane xy

    vx_1, vy_1 = Dy(psi_1) , -Dx(psi_1)


    #Velocities with Stream function in plane xz

    vx_2, vz_2 = Dz(psi_2) , -Dx(psi_2)


    #Velocities with Stream function in plane zy

    vy_3, vz_3 = Dz(psi_3) , -Dy(psi_3)


    #From just eddies
    vx=vx_1+vx_2
    vy=vy_1+vy_3
    vz=vz_2+vz_3


    v = np.sqrt(vx**2+vy**2+vz**2)


    # Adjust s parameter so that the max velocity is vmax
    mult = params.vmax / v.max()
    vx = vx*mult
    vy = vy*mult
    vz = vz*mult


    # Calculate velocity derivatives to use in calculations
    #dv = Dx(vx) + Dy(vy) + Dz(vz)
    dv = Dx(vx) + Dy(vy) + Dz_Neumann(vz)
    
    print("Max vz bottom:", np.max(np.abs(vz[:, :, 0])))  
    print("Max vz top :", np.max(np.abs(vz[:, :, -1])))    
    print("Max dv :", np.max(np.abs(dv[:, :, :]))) 
    #print("Max dv_neumann :", np.max(np.abs(dv2[:, :, :])))
    
    plt.figure(figsize=(5,5))
    plt.scatter(eddy.x, eddy.z, c=eddy.s_y.flatten())  # Color s_y 
    plt.axhline(y=domain.z_low, color='r', linestyle='--')  
    plt.axhline(y=domain.z_up, color='r', linestyle='--')
    plt.axvline(x=domain.x_low, color='r', linestyle='--')
    plt.axvline(x=domain.x_up, color='r', linestyle='--')
    plt.xlabel('x (km)', fontsize=13, color='k')
    plt.ylabel('z (km)', fontsize=13, color='k')
    plt.title('Posiciones Remolinos (Rotaciones en y)', fontsize=18, pad=8.5, color='k')
    plt.tick_params(axis='both', colors='k')  
    plt.savefig("NOPeriodEddiesXZNegro.png", dpi=150, bbox_inches="tight")
    plt.show()

    plt.figure(figsize=(5,5))    
    plt.scatter(eddy.y, eddy.z, c=eddy.s_x.flatten())  # Color s_x 
    plt.axhline(y=domain.z_low, color='r', linestyle='--')  
    plt.axhline(y=domain.z_up, color='r', linestyle='--')
    plt.axvline(x=domain.y_low, color='r', linestyle='--')
    plt.axvline(x=domain.y_up, color='r', linestyle='--')
    plt.xlabel('y (km)', fontsize=13, color='k')
    plt.ylabel('z (km)', fontsize=13, color='k')
    plt.title('Posiciones Remolinos (Rotaciones en x)', fontsize=18, pad=8.5, color='k')
    plt.tick_params(axis='both', colors='k')  
    plt.savefig("NOPeriodEddiesYZNegro.png", dpi=150, bbox_inches="tight")
    plt.show()    

    
    
    
#     plt.plot(psi_2[10, 2, :],z[0, 0, :] )  
#     plt.axhline(y=domain.z_low, color='r', linestyle='--')
#     plt.axhline(y=domain.z_up, color='r', linestyle='--')
#     plt.xlabel('psi_2'); plt.ylabel('z')
#     plt.title('Psi 2 boundareis')
#     plt.show()
    
#     plt.plot(psi_3[10, 2, :],z[0, 0, :] )  
#     plt.axhline(y=domain.z_low, color='r', linestyle='--')
#     plt.axhline(y=domain.z_up, color='r', linestyle='--')
#     plt.xlabel('psi_3'); plt.ylabel('z')
#     plt.title('Psi 3 boundaries')
#     plt.show()

    return vx, vy, vz , dv

###  Generador de turbulencias

In [ ]:
vx, vy, vz, dv = GenEddy()





In [ ]:
   # Should be ~0

In [ ]:
# Mount Google Drive to Colab
#from google.colab import drive
#drive.mount('/content/drive')
# Guardar en Drive
np.save("vx_noperiod.npy", vx)
np.save("vy_noperiod.npy", vy)
np.save("vz_noperiod.npy", vz)
np.save("dv_noperiod.npy", dv)

### **Patches**


In [ ]:
def patches(domain, tspan, params, eddy, iCondN, iCondP, iCondType, GenEddys,NombreSim, RecordStep):
    ######PARAMETERS######


  tspan= params.mu*tspan   #paso directamente a tiempo adimensional

  dt = tspan[1]     #Time step


  # Create time vector (adimensional ya)
  t = np.arange(tspan[0], tspan[2] + dt, dt)    #El +dt es necesario para que llegue a tspan[2]
                                                #T es el tiempo en general en días, se revierte al final dividiendo t por params.mu
  x, y, z = np.meshgrid(np.linspace(domain.x_low, domain.x_up, domain.x_cells),
                        np.linspace(domain.y_low, domain.y_up, domain.y_cells),
                        np.linspace(domain.z_low, domain.z_up, domain.z_cells),
                        indexing='ij')





  IN = params.IN / (params.mu * params.HN)   # Dimensionless input rate of nutrients 0.9, 0.32, 0.38
  a  = params.k * params.HP / (params.HN)    # Dimensionsless nutrient content in phytoplankton 0.8
  mn = params.mN / params.mu                 # Dimensionless loss rate of nutrient 0.03
  fp = params.fP / (params.mu * params.HP)   # Dimensionless max feeding rate of zoo on phyto 0.9


  # Initialize variables
  n_1 = np.zeros(x.shape) # Dimensionless nutrient concentration (inicial)
  n_2 = np.zeros(x.shape) # Dimensionless nutrient concentration (final)
  p_1 = np.zeros(x.shape) # Dimensionless phytoplankton concentration (inicial)
  p_2 = np.zeros(x.shape) # Dimensionless phytoplankton concentration (final)

  def DnFun(n,p):
  # dn = d*(DDx(n)+DDy(n)) - n.*dv - (vx.*Dx(n)+vy.*Dy(n)) + in - a*n./(1+n).*p - mn*n;  Esto es en 2D
    dn = 0;
    dn = dn - n*dv - (vx*Dx(n)+vy*Dy(n)+vz*Dz_Neumann(n))              # Advection
    dn = dn + params.d*(DDx(n)+DDy(n)+DDz_Neumann(n))                  # Diffusion
    dn = dn + IN - a*n/(1+n)*p - mn*n                          # Reaction

    return dn

  def DpFun(n,p):
  #    dp = d*(DDx(p)+DDy(p)) - p.*dv - (vx.*Dx(p)+vy.*Dy(p)) + n./(1+n).*p - fp*p./(1+p); Esto es en 2D
    dp = 0
    dp = dp - p*dv - (vx*Dx(p)+vy*Dy(p)+vz*Dz_Neumann(p))                          # Advection
    dp = dp + params.d*(DDx(p)+DDy(p)+DDz_Neumann(p))                              # Diffusion
    dp = dp + n/(1+n)*p - fp*p/(1+p)                                       # Reaction



    return dp

  def adjustIC(iCondN, iCondP):
    err = np.zeros((2, 2))              ######### BUSCAR BIEN QUE AJUSTE SE HACE, POR SI ACASO SE
    c1 = 1
    c2 = (a*fp-IN-a+mn)/mn
    c3 = (a*fp-IN)/mn
    n1 = (-c2+math.sqrt(c2**2-4*c1*c3))/(2*c1)
    p1 = fp/n1+fp-1;

    if n1 < 0 or p1 < 0:
      err[0][:]= [n1,p1]
      n1 = (-c2-math.sqrt(c2**2-4*c1*c3))/(2*c1)
      p1 = fp/n1+fp-1

    if n1 < 0 or p1 < 0:
      err[1][:]= [n1,p1]
      #print('err')
      #raise ValueError('The given parameters do not have a valid stationary point')
      warnings.warn('The given parameters do not have a valid stationary point')
    #Transform to dimensionless space
    iCondn = iCondN / params.HN
    iCondp = iCondP / params.HP

    #Translate so that the mean is centered at the stationary point
    iCondn = iCondn + n1 - np.mean(iCondn)
    iCondp = iCondp + p1 - np.mean(iCondp)

    #Scale range keeping the mean if there are negative values
    if np.any(iCondn <= 0):
      iCondn = (iCondn - n1)/(n1-np.min(iCondn))*(params.A*n1)+n1

    if np.any(iCondp <= 0):
      iCondp = (iCondp - p1)/(p1-np.min(iCondp))*(params.A*p1)+p1

    # Display adjustment in physical space
    iCondN = iCondn * params.HN; iCondP = iCondp * params.HP;
    print(f"iCondN adjusted to [{np.min(iCondN):.6f}, {np.max(iCondN):.6f}] with mean {np.mean(iCondN):.6f}")
    print(f"iCondP adjusted to [{np.min(iCondP):.6f}, {np.max(iCondP):.6f}] with mean {np.mean(iCondP):.6f}")

    # Error checking
    if abs(np.mean(iCondn) - n1) > 1e-10:
      warnings.warn(f'Something went wrong in adjustment. Mean of iCondn does not match stationary point n1 ({np.mean(iCondn):.6f} vs. {n1:.6f})')

    if abs(np.mean(iCondp) - p1) > 1e-10:
      warnings.warn(f'Something went wrong in adjustment. Mean of iCondp does not match stationary point p1 ({np.mean(iCondp):.6f} vs. {p1:.6f})')

    return iCondn, iCondp


  def RK4(n,p):

    k1 = dt*DnFun(n,p)
    m1 = dt*DpFun(n,p)

    k2 = dt*DnFun(n+k1/2, p+m1/2)
    m2 = dt*DpFun(n+k1/2, p+m1/2)

    k3 = dt*DnFun(n+k2/2, p+m2/2)
    m3 = dt*DpFun(n+k2/2, p+m2/2)

    k4 = dt*DnFun(n+k3, p+m3)
    m4 = dt*DpFun(n+k3, p+m3)

    n = n + (k1+(k2*2)+(k3*2)+k4)/6
    p = p + (m1+(m2*2)+(m3*2)+m4)/6

    return n,p

  # Initial conditions
  # iCondType is given and is 'adjust'

  if str(iCondType).lower() == "adjust":
    # Adjust initial conditions and assign to dimensionless conditions
    iCondn, iCondp = adjustIC(iCondN, iCondP)
    n_1[:,:,:] = iCondn
    p_1[:,:,:] = iCondp

    # iCondType is not given or is not 'adjust'
  else:
    # Assign initial dimensionless conditions
    n_1[:,:,:] = iCondN / params.HN
    p_1[:,:,:] = iCondP / params.HP


    # Error detection. Concentrations should not go below zero. Check the given parameters.
  if np.any(n_1[:,:,:] < 0):
    warnings.warn('The initial conditions for N (iCondN) has negative concentrations. Consider adjusting the input parameters.')

  if np.any(p_1[:,:,:] < 0):
    warnings.warn('The initial conditions for P (iCondP) has negative concentrations. Consider adjusting the input parameters.')

  if str(GenEddys).lower() == "generate":
    vx, vy, vz, dv = GenEddy()

  else:
     vx=np.load(f"vx_{str(GenEddys).lower()}.npy")
     vy=np.load(f"vy_{str(GenEddys).lower()}.npy")
     vz=np.load(f"vz_{str(GenEddys).lower()}.npy")
     dv=np.load(f"dv_{str(GenEddys).lower()}.npy")
########CALCULATIONS##############

  #print('Time steps:   0%%')
  tN = len(t)
  step=0
  ruta = "datos.h5"  # Ruta dentro de Google Drive

  with h5py.File(ruta, "a") as f:  # "a" = append (añadir sin borrar)
    if str(NombreSim).lower() in f:          #Check group 
        del f[str(NombreSim).lower()]        
    grupo = f.require_group(str(NombreSim).lower())
    grupo.create_dataset(f"Cianobacteria_{0}", data=p_1* params.HP)
    grupo.create_dataset(f"Nutrientes_{0}", data=n_1* params.HN)
    grupo.create_dataset(f"Tau{0}",data=np.array(0))
    for tau in range(1, tN): #primera 1, último tN-1
      if tau%(tN//100)==0:
        print(f"\r{round(tau/tN*100):3}%", end="", flush=True)


        # Runge-Kutta 4
      nTemp, pTemp = RK4(n_1[:, :,:], p_1[:, :, :])
      n_2[:, :,:] = nTemp
      p_2[:, :,:] = pTemp

      if tau%RecordStep==0:
        step+=1
        grupo.create_dataset(f"Cianobacteria_{tau}", data=p_2* params.HP)
        grupo.create_dataset(f"Nutrientes_{tau}", data=n_2* params.HN)
        grupo.create_dataset(f"Tau{step}",data=tau)

      n_1 = n_2
      p_1 = p_2

    grupo.create_dataset("Step",data=np.array([step+1,float(tSpan[1])]))




  print("\r100%")

    # Error detection. Concentrations should not go below zero. Check the given parameters.
  #if np.any(n < 0):
        #warnings.warn('The solution for N has negative concentrations. Consider adjusting the input parameters.');

  #if np.any(p < 0):
        #warnings.warn('The solution for P has negative concentrations. Consider adjusting the input parameters.');


    # Transform dimensionless representation to physical
  #N = n * params.HN
  #P = p * params.HP
  #T  = t/params.mu


  print('Done!')

  return

### Inicio Simulación

In [ ]:
T = patches(domain, tSpan, params, eddy, iCondN, iCondP,'adjust','noperiod',NombreSim,RecordStep)

### Funciones Visualizado

In [ ]:
ruta = "finaldata.h5"


with h5py.File(ruta, "r") as f:
    grupo = f[str(NombreSim).lower()]
    STEP = grupo["Step"][0] 
    tSpan = grupo["Step"][1]
    Tau = [np.array(grupo[f"Tau{step}"]) for step in range(int(STEP))]
    cianobacterias = [np.array(grupo[f"Cianobacteria_{tau}"]) for tau in Tau]
    nutrientes = [np.array(grupo[f"Nutrientes_{tau}"]) for tau in Tau]

In [ ]:
print(STEP)
cianobacterias_arr = np.stack(cianobacterias, axis=0) 
print(cianobacterias_arr.shape)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  

x_flat = x.flatten()
y_flat = y.flatten()
z_flat = z.flatten()

t = 0

out_dir = "ModeloNoPeriodico"
os.makedirs(out_dir, exist_ok=True)

vmin, vmax = 0, cianobacterias_arr[t].max()

fig = plt.figure(figsize=(6, 5), constrained_layout=True)
ax = fig.add_subplot(111, projection='3d')

ax.set_xlim(domain.x_low, domain.x_up)
ax.set_ylim(domain.y_low, domain.y_up)
ax.set_zlim(domain.z_low, domain.z_up)
ax.set_xlabel("x (km)", fontsize=13)
ax.set_ylabel("y (km)", fontsize=13)
ax.set_zlabel("z (km)", fontsize=13)
ax.tick_params(axis='both', which='major', labelsize=10)
ax.tick_params(axis='z', which='major', labelsize=10)

densities = cianobacterias_arr[t].flatten()
sc = ax.scatter(
    x_flat, y_flat, z_flat,
    c=densities,
    cmap='viridis',
    s=5,
    vmin=vmin, vmax=vmax
)

day = t * tSpan
ax.set_title(f"Día {day:.2f}", fontsize=18)

cbar = fig.colorbar(
    sc,
    ax=ax,
    orientation='vertical',
    shrink=0.8,
    aspect=20,
    pad=0.05
)
cbar.ax.set_ylabel('Concentración ($g/m^3$)', rotation=90, labelpad=10, fontsize=13)
cbar.ax.tick_params(labelsize=10)

out_path = os.path.join(out_dir, f"CIANO_INITIAL_{t}NP.png")
plt.savefig(out_path, dpi=150, bbox_inches='tight', transparent=True)
plt.close(fig)

print(f"Guardado {out_path}")

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  



x_flat = x.flatten()
y_flat = y.flatten()
z_flat = z.flatten()


times_groups = [
    [100, 200, 300, 400],
    [500, 600, 700, 800],
    [900, 1000, 1100, 1200],
]
#     [0, 100, 200, 300],
#     [400, 500, 600, 700],
#     [800, 900, 1000, 1100],
#     [0, 50, 100, 150],
#     [200, 250, 300, 350],
#     [400, 450, 500, 550],

out_dir = "ModeloNoPeriodico"
os.makedirs(out_dir, exist_ok=True)

for idx, times in enumerate(times_groups, start=1):
    #calcular vmax para este grupo de tiempos

    group_max = max(cianobacterias_arr[t].max() for t in times)
    vmin, vmax = 0, group_max

    
    fig, axs = plt.subplots(
        nrows=1, ncols=4,
        figsize=(18, 4),
        subplot_kw={'projection': '3d'},
        constrained_layout=True
    )

    for ax, t in zip(axs, times):

        ax.set_xlim(domain.x_low, domain.x_up)
        ax.set_ylim(domain.y_low, domain.y_up)
        ax.set_zlim(domain.z_low, domain.z_up)
        ax.set_xlabel("x (km)", fontsize=13)
        ax.set_ylabel("y (km)", fontsize=13)
        ax.set_zlabel("z (km)", fontsize=13)
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.tick_params(axis='z', which='major', labelsize=10)


        densities = cianobacterias_arr[t].flatten()
        sc = ax.scatter(
            x_flat, y_flat, z_flat,
            c=densities,
            cmap='viridis',
            s=5,
            vmin=vmin, vmax=vmax
        )


        day = t * tSpan
        ax.set_title(f"Día {day:.2f}", fontsize=18)

    # Colorbar común
    cbar = fig.colorbar(
        sc,
        ax=axs,
        orientation='vertical',
        shrink=0.8,
        aspect=20,
        pad=0.05
    )
    cbar.ax.set_ylabel('Concentración ($g/m^3$)', rotation=90, labelpad=10, fontsize=13)
    cbar.ax.tick_params(labelsize=10)

    # Guardar figura
    out_path = os.path.join(out_dir, f"quadCIANO_frame{times}NP.png")
    plt.savefig(out_path, dpi=150, bbox_inches='tight',transparent=True)
    plt.close(fig)

    print(f"Guardado {out_path}")


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 

#  plano de corte: ax + by + cz + d = 0


a, b, c, d = 0, 1, -1, 0  # cuña que siempre muesta x, secciona y-z
# a, b, c, d = -1, +1, 0, 0  # cuña que siempre muesta z, secciona y-x
# a, b, c, d =- 0.5, 0.5, -1, 0  # cuña que siempre muesta x e y

# funcion que dice si un punto esta por encima del plano
def is_on_side(x, y, z):
    return (a * x + b * y + c * z + d) >= 0


times_groups = [
    [100, 200, 300, 400],
    [500, 600, 700, 800],
    [900, 1000, 1100, 1200],
]


out_dir = "ModeloNoPeriodico"
os.makedirs(out_dir, exist_ok=True)

for idx, times in enumerate(times_groups, start=1):
    # Calcular vmax para este grupo de tiempos
    group_max = max(cianobacterias_arr[t].max() for t in times)
    vmin, vmax = 0, group_max


    fig, axs = plt.subplots(
        nrows=1, ncols=4,
        figsize=(18, 4),
        subplot_kw={'projection': '3d'},
        constrained_layout=True
    )

    for ax, t in zip(axs, times):
        ax.set_xlim(domain.x_low, domain.x_up)
        ax.set_ylim(domain.y_low, domain.y_up)
        ax.set_zlim(domain.z_low, domain.z_up)
        ax.set_xlabel("x (km)", fontsize=13)
        ax.set_ylabel("y (km)", fontsize=13)
        ax.set_zlabel("z (km)", fontsize=13)
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.tick_params(axis='z', which='major', labelsize=10)

        # filtrar puntos por plano de corte
        mask = is_on_side(x_flat, y_flat, z_flat)
        xs, ys, zs = x_flat[mask], y_flat[mask], z_flat[mask]
        densities = cianobacterias_arr[t].flatten()[mask]


        sc = ax.scatter(
            xs, ys, zs,
            c=densities,
            cmap='viridis',
            s=5,
            vmin=vmin, vmax=vmax
        )

        day = t * tSpan
        ax.set_title(f"Día {day:.2f}", fontsize=18)

    # Colorbar común
    cbar = fig.colorbar(
        sc,
        ax=axs,
        orientation='vertical',
        shrink=0.8,
        aspect=20,
        pad=0.05
    )
    cbar.ax.set_ylabel('Concentración ($g/m^3$)', rotation=90, labelpad=10,fontsize=13)
    cbar.ax.tick_params(labelsize=10)

    # Guardar figura
    out_path = os.path.join(out_dir, f"quadCIANO_frame{times,a,b,c,d}NP.png")
    plt.savefig(out_path, dpi=150, bbox_inches='tight',transparent=True)
    plt.close(fig)

    print(f"Guardado {out_path}")

# Nutrientes


In [ ]:
nutrientes_arr = np.stack(nutrientes, axis=0) 
print(nutrientes_arr.shape)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  

x_flat = x.flatten()
y_flat = y.flatten()
z_flat = z.flatten()

t = 0

out_dir = "ModeloNoPeriodico"
os.makedirs(out_dir, exist_ok=True)

vmin, vmax = 0, nutrientes_arr[t].max()

fig = plt.figure(figsize=(6, 5), constrained_layout=True)
ax = fig.add_subplot(111, projection='3d')

ax.set_xlim(domain.x_low, domain.x_up)
ax.set_ylim(domain.y_low, domain.y_up)
ax.set_zlim(domain.z_low, domain.z_up)
ax.set_xlabel("x (km)", fontsize=13)
ax.set_ylabel("y (km)", fontsize=13)
ax.set_zlabel("z (km)", fontsize=13)
ax.tick_params(axis='both', which='major', labelsize=10)
ax.tick_params(axis='z', which='major', labelsize=10)

densities = nutrientes_arr[t].flatten()
sc = ax.scatter(
    x_flat, y_flat, z_flat,
    c=densities,
    cmap='cividis',
    s=5,
    vmin=vmin, vmax=vmax
)

day = t * tSpan
ax.set_title(f"Día {day:.2f}", fontsize=18)

cbar = fig.colorbar(
    sc,
    ax=ax,
    orientation='vertical',
    shrink=0.8,
    aspect=20,
    pad=0.05
)
cbar.ax.set_ylabel('Concentración ($g/m^3$)', rotation=90, labelpad=10, fontsize=13)
cbar.ax.tick_params(labelsize=10)

out_path = os.path.join(out_dir, f"NUTRI_INITIAL_{t}NP.png")
plt.savefig(out_path, dpi=150, bbox_inches='tight', transparent=True)
plt.close(fig)

print(f"Guardado {out_path}")

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  



x_flat = x.flatten()
y_flat = y.flatten()
z_flat = z.flatten()


times_groups = [
    [100, 200, 300, 400],
    [500, 600, 700, 800],
    [900, 1000, 1100, 1200],
]
#     [0, 100, 200, 300],
#     [400, 500, 600, 700],
#     [800, 900, 1000, 1100],
#     [0, 50, 100, 150],
#     [200, 250, 300, 350],
#     [400, 450, 500, 550],

out_dir = "ModeloNoPeriodico"
os.makedirs(out_dir, exist_ok=True)

for idx, times in enumerate(times_groups, start=1):
    # Calcular vmax para este grupo de tiempos

    group_max = max(nutrientes_arr[t].max() for t in times)
    vmin, vmax = 0, group_max


    fig, axs = plt.subplots(
        nrows=1, ncols=4,
        figsize=(18, 4),
        subplot_kw={'projection': '3d'},
        constrained_layout=True
    )

    for ax, t in zip(axs, times):
        # Ajustes de ejes
        ax.set_xlim(domain.x_low, domain.x_up)
        ax.set_ylim(domain.y_low, domain.y_up)
        ax.set_zlim(domain.z_low, domain.z_up)
        ax.set_xlabel("x (km)", fontsize=13)
        ax.set_ylabel("y (km)", fontsize=13)
        ax.set_zlabel("z (km)", fontsize=13)
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.tick_params(axis='z', which='major', labelsize=10)


        densities = nutrientes_arr[t].flatten()
        sc = ax.scatter(
            x_flat, y_flat, z_flat,
            c=densities,
            cmap='cividis',
            s=5,
            vmin=vmin, vmax=vmax
        )


        day = t * tSpan
        ax.set_title(f"Día {day:.2f}", fontsize=18)

    # Colorbar comun
    cbar = fig.colorbar(
        sc,
        ax=axs,
        orientation='vertical',
        shrink=0.8,
        aspect=20,
        pad=0.05
    )
    cbar.ax.set_ylabel('Concentración ($g/m^3$)', rotation=90, labelpad=10, fontsize=13)
    cbar.ax.tick_params(labelsize=10)

    # Guardar figura
    out_path = os.path.join(out_dir, f"quadNUTRI_frame{times}NP.png")
    plt.savefig(out_path, dpi=150, bbox_inches='tight',transparent=True)
    plt.close(fig)

    print(f"Guardado {out_path}")

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 

#  plano de corte: ax + by + cz + d = 0


a, b, c, d = 0, 1, -1, 0  # cuña que siempre muesta x, secciona y-z
# a, b, c, d = -1, +1, 0, 0  # cuña que siempre muesta z, secciona y-x
# a, b, c, d =- 0.5, 0.5, -1, 0  # cuña que siempre muesta x e y

# funcion que decide si un punto esta por encima del plano
def is_on_side(x, y, z):
    return (a * x + b * y + c * z + d) >= 0


times_groups = [
    [100, 200, 300, 400],
    [500, 600, 700, 800],
    [900, 1000, 1100, 1200],
]


out_dir = "ModeloNoPeriodico"
os.makedirs(out_dir, exist_ok=True)

for idx, times in enumerate(times_groups, start=1):
    # Calcular vmax para este grupo de tiempos
    group_max = max(nutrientes_arr[t].max() for t in times)
    vmin, vmax = 0, group_max


    fig, axs = plt.subplots(
        nrows=1, ncols=4,
        figsize=(18, 4),
        subplot_kw={'projection': '3d'},
        constrained_layout=True
    )

    for ax, t in zip(axs, times):
        ax.set_xlim(domain.x_low, domain.x_up)
        ax.set_ylim(domain.y_low, domain.y_up)
        ax.set_zlim(domain.z_low, domain.z_up)
        ax.set_xlabel("x (km)", fontsize=13)
        ax.set_ylabel("y (km)", fontsize=13)
        ax.set_zlabel("z (km)", fontsize=13)
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.tick_params(axis='z', which='major', labelsize=10)

        # filtrar puntos por plano de corte
        mask = is_on_side(x_flat, y_flat, z_flat)
        xs, ys, zs = x_flat[mask], y_flat[mask], z_flat[mask]
        densities = nutrientes_arr[t].flatten()[mask]


        sc = ax.scatter(
            xs, ys, zs,
            c=densities,
            cmap='cividis',
            s=5,
            vmin=vmin, vmax=vmax
        )

        day = t * tSpan
        ax.set_title(f"Día {day:.2f}", fontsize=18)

    # Colorbar común
    cbar = fig.colorbar(
        sc,
        ax=axs,
        orientation='vertical',
        shrink=0.8,
        aspect=20,
        pad=0.05
    )
    cbar.ax.set_ylabel('Concentración ($g/m^3$)', rotation=90, labelpad=10,fontsize=13)
    cbar.ax.tick_params(labelsize=10)

    # Guardar figura
    out_path = os.path.join(out_dir, f"quadNUTRI_frame{times,a,b,c,d}NP.png")
    plt.savefig(out_path, dpi=150, bbox_inches='tight',transparent=True)
    plt.close(fig)

    print(f"Guardado {out_path}")

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

def VisualizadorFondoSuperficie(N, t,n_imag):

    out_dir = "ModeloNoPeriodico"
    os.makedirs(out_dir, exist_ok=True)

    # fondo (0) y superficie (-1)
    h = [0, -1]

    z_legend = np.linspace(domain.z_low, domain.z_up, domain.z_cells)

    # global vmin/vmax
    planes = [N[t, :, :, corte] for corte in h]
    vmin = min(p.min() for p in planes)
    vmax = max(p.max() for p in planes)


    fig, axes = plt.subplots(1, n_imag, figsize=(5*n_imag, 5), constrained_layout=True)
    day = t * tSpan

    for ax, N_plane, corte in zip(axes, planes, h):
        im = ax.imshow(
            N_plane,
            extent=[x.min(), x.max(), y.min(), y.max()],
            origin='lower',
            vmin=vmin,
            vmax=vmax
        )
        ax.set_xlabel("x (km)", fontsize=13)
        ax.set_ylabel("y (km)", fontsize=13)
        #which='major' Solo modifica los ticks principales 
        ax.tick_params(axis='both', which='major', labelsize=8)
        ax.set_title(f'Plano xy en z = {z_legend[corte]:.2f} km \npara T = Día {t*tSpan:.2f} ')

    # Colorbar para ambos
    
    cbar = fig.colorbar(im, ax=axes, shrink=0.8, aspect=20, pad=0.1)
                       #shrink=longbarra, aspect=prop.largoancho, pad=dist al plot
    cbar.ax.set_ylabel('Concentración ($g/m^3$)', rotation=90, labelpad=8)
                                         # labelpad=Espacia la etiqueta
    cbar.ax.tick_params(labelsize=8)

    # Guardado
    filename = f"FondoSuperficiePeriod_Día{day:.2f}NegroNP.png"
    out_path = os.path.join(out_dir, filename)
    fig.savefig(out_path, dpi=150, bbox_inches='tight')
    plt.close(fig)

    print(f"Guardado {out_path}")


In [ ]:

for t in np.arange(0,400,20):
    VisualizadorFondoSuperficie(cianobacterias_arr,t,n_imag=2)